# TD - Analyse des données de parcours et réussite des bacheliers en licence

Ce notebook analyse les données du fichier `fr-esr-parcours-et-reussite-des-bacheliers-en-licence.csv`.


## 1. Import des bibliothèques et chargement des données


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
# Import du fichier CSV avec le séparateur point-virgule
df = pd.read_csv('fr-esr-parcours-et-reussite-des-bacheliers-en-licence.csv', 
                 sep=';', 
                 encoding='utf-8')


## 2. Exploration initiale des données


In [ ]:
# Afficher les premières lignes, dimensions et noms des colonnes pour vérifier l'import
print(df.head())
print(f"\nDimensions du DataFrame : {df.shape}")
print(f"\nNoms des colonnes :\n{df.columns.tolist()}")


## 3. Nettoyage et préparation des données


In [ ]:
# Ne garder que toutes les colonnes sauf les 5 dernières (statistiques L3 non utiles ici)
df = df.iloc[:, :-5]

# Colonnes de filière (6 premières colonnes)
filiere_cols = [
    'Id Grande discipline', 'Grande discipline',
    'Id Discipline', 'Discipline',
    'Id Secteur disciplinaire', 'Secteur disciplinaire'
]

print(f"Colonnes de filière identifiées : {filiere_cols}")
print(f"Nouvelles dimensions après nettoyage : {df.shape}")


## 4. Vérification des valeurs manquantes


In [ ]:
# 1) Vérifier les valeurs manquantes (NaN) et chaînes vides/blanches
print("Valeurs manquantes par colonne:")
print(df[filiere_cols].isna().sum())


In [ ]:
# Compter aussi les vides/blancs côté intitulés
def is_blank(s):
    return s.astype(str).str.strip().eq('')

for col in ['Grande discipline', 'Discipline', 'Secteur disciplinaire']:
    print(f"Nb de chaînes vides/blanches dans {col}: {is_blank(df[col]).sum()}")


## 5. Normalisation des données


In [ ]:
# 2) Normaliser légèrement les intitulés pour éviter les faux doublons (espaces)
for col in ['Grande discipline', 'Discipline', 'Secteur disciplinaire']:
    df[col] = df[col].astype(str).str.strip()

print("Normalisation des espaces effectuée pour les colonnes d'intitulés.")


## 6. Vérification de l'unicité des identifiants


In [ ]:
# 3) Vérifier l'unicité: chaque ID doit correspondre à un seul intitulé
pairs = [
    ('Id Grande discipline', 'Grande discipline'),
    ('Id Discipline', 'Discipline'),
    ('Id Secteur disciplinaire', 'Secteur disciplinaire')
]

for id_col, label_col in pairs:
    nunique_per_id = df.groupby(id_col)[label_col].nunique(dropna=False).sort_values(ascending=False)
    conflits_id = nunique_per_id[nunique_per_id > 1]
    print(f"\nVérification {id_col} -> {label_col}:")
    if conflits_id.empty:
        print("OK: chaque ID correspond à un seul intitulé.")
    else:
        print("Conflits (un même ID a plusieurs intitulés):")
        print(conflits_id.head(20))


In [ ]:
# Vérifier aussi l'inverse (un intitulé ↔ plusieurs IDs)
for id_col, label_col in pairs:
    nunique_per_label = df.groupby(label_col)[id_col].nunique(dropna=False).sort_values(ascending=False)
    conflits_label = nunique_per_label[nunique_per_label > 1]
    print(f"\nVérification inverse {label_col} -> {id_col}:")
    if conflits_label.empty:
        print("OK: chaque intitulé correspond à un seul ID.")
    else:
        print("Conflits (un même intitulé a plusieurs IDs):")
        print(conflits_label.head(20))


## 7. Résumé final des colonnes


In [ ]:
# 4) Afficher un résumé des colonnes pour vérification
print("\nInfo des 6 colonnes de filière après contrôles:")
print(df[filiere_cols].info())
